### Calculating the elctirical energy demand for energy positive building.

In [9]:
import numpy as np
import random
#import ramp

### Before starting we are assuming few things.
### The method is using different equipment behaviour we create a load profile for a building and assume the PV and wind system to fullfill the demand.



### Room behavior
## Equipment Used
## - Light, power socket - heatpump  

In [57]:
# Calcluating the load profile of the building
class Energymodel:
    def __init__(self,rooms):
        self.total_rooms = rooms
        self.appliances = {
            "ceiling_light": 0.055, # per light
            "sockets": 0.1,  # 100 er for mobile use or other
            "heat_pump": 0.225,  # 0.225kW per heat pump (for heating & cooling)  0.015kw/m^2
            "common_area_ligths" : 0.065 # 65W 
        }
        
        self.seasonal_usage_hours = {
            "summer": {
                "ceiling_light": (2, 6), "sockets": (2, 4), "heat_pump": (1, 3), "common_area_ligths" :(4, 6) #summer usage approx
            },
            "winter": {
                "ceiling_light": (5, 8), "sockets": (2, 4), "heat_pump": (4, 10), "common_area_ligths" :(6, 8) #winter usage approx
            }
        }    
        
        self.appliances_count = {
            "ceiling_light": 4,
            "sockets": 4,
            "heat_pump": 1,
            "common_area_ligths" : 18
        }

    def get_utilized_rooms(self):       
        # 10% chance that the building is completely empty
        if random.random() < 0.2:  
            return 0
        # 40% chance that only 4-5 rooms are occupied (off-season)
        elif random.random() < 0.4:  
            return random.randint(4, 5)
        # Otherwise, full occupancy
        else:
            return self.total_rooms
        
    def calculate_daily_consumption(self,season):
        use_hours_season = self.seasonal_usage_hours[season]
        occupied_rooms = self.get_utilized_rooms()
        daily_usage = 0 # excluding common lights
        Common_light = 0
        for appliance, power in self.appliances.items():
            # Generating random usage time within the given range
            random_hours = random.uniform(use_hours_season[appliance][0],use_hours_season[appliance][1])
            if appliance == "common_area_ligths":
                Common_light += power * random_hours * self.appliances_count[appliance]
                continue
            daily_usage += power * random_hours * self.appliances_count[appliance]
        daily_final = (daily_usage * occupied_rooms) + Common_light # Adjusting based on room occupancy
        return daily_final
    

In [58]:
rooms = int(input("Enter number of rooms: ") or 10)


model = Energymodel(rooms)



#total_summer_demand = 0
#total_winter_demand = 0

summer_days = 180
winter_days = 185

# to make it fast we use numpy

# for day in range(365):
#     if day < 180:  # Assume first half of the year is summer
#         total_summer_demand += model.calculate_daily_consumption(day, "summer")
#     else:  # Second half of the year is winter
#         total_winter_demand += model.calculate_daily_consumption(day, "winter")

total_summer_demand = np.sum([model.calculate_daily_consumption("summer") for _ in range(summer_days)])
total_winter_demand = np.sum([model.calculate_daily_consumption("winter") for _ in range(winter_days)])

avg_summer_demand = total_summer_demand / 180
avg_winter_demand = total_winter_demand / 185

In [59]:
print(f" Average summer demand {total_summer_demand} and Averagee winter demand {total_winter_demand}")

 Average summer demand 3709.7160780447293 and Averagee winter demand 6322.341555447833


### calculating the PV and wind outputs - renewable outputs

In [21]:
PV_capacity = 13 # float(input("enter PV_capacity in kW"))
wind_capacity = 20
# https://en.climate-data.org/europe/germany/saxony-anhalt/ilsenburg-harz-22054/  
sunlight_hours = {"summer": 8, "winter": 3.5}
# https://weatherspark.com/y/67815/Average-Weather-in-Ilsenburg-Saxony-Anhalt-Germany-Year-Round
wind_speed = {"summer": 9, "winter": 12}
PV_efficiency = 0.8 # (18 -19 %)Recent Facts about Photovoltaics in Germany Fraunhofer
wind_efficiency = 0.35 # https://www.epa.gov/sites/default/files/2019-08/documents/wind_turbines_fact_sheet_p100il8k.pdf

def pv_output(capacity, efficiency, hours):
    output = capacity * hours * efficiency
    return output


def wind_output(speed, efficiency):
    air_density = 1.225  # kg/m^3  https://www.emd-international.com/files/windpro/WindPRO_Power_Curve_Options.pdf
    swept_area = 63.6 #m^2  -for 20kW,  https://www.watulagreentech.fi/en/wind-turbines/wind-turbine-20kw-system-horizontal/
    wind_power = 0.5 * air_density * swept_area * (speed ** 3) * efficiency # https://www.e-education.psu.edu/emsc297/node/649
    return (wind_power * 24 / 1000) #  daily wind energy generation in kWh

In [22]:
wind_yearly = (wind_output(wind_speed["summer"], wind_efficiency)) * summer_days  + (wind_output(wind_speed["winter"], wind_efficiency))*winter_days
Pv_yearly = (pv_output(PV_capacity, PV_efficiency, sunlight_hours["summer"])) * summer_days + (pv_output(PV_capacity, PV_efficiency, sunlight_hours["winter"])) *winter_days


In [23]:
print(f"Yearly output of wind turbine = {wind_yearly} and Yearly output of solar = {Pv_yearly} ")

Yearly output of wind turbine = 147544.3998 and Yearly output of solar = 21710.0 


In [24]:
summer_output = pv_output(PV_capacity, PV_efficiency, sunlight_hours["summer"]) # + wind_output(wind_speed["summer"], wind_efficiency)
winter_output = pv_output(PV_capacity, PV_efficiency, sunlight_hours["winter"])  #+ wind_output(wind_speed["winter"], wind_efficiency)

In [25]:
summer_output * summer_days

14976.0

In [26]:
winter_output * winter_days

6734.0